In [1]:
# Assumes Anaconda Distribution

# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

# Backoff time for large scrapes
THROTTLE_TIME = .05

# ipython magics
%load_ext watermark
%matplotlib inline
# a nice bonus for making progress bars in ipython
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
tqdm.pandas()

In [14]:
!ls "../data/processed"

high_schools_processed.csv
player_bios_processed.csv
player_bios_processed_wrapped.csv
rosters_processed.csv


In [36]:
procPath = os.path.join(PROJ_ROOT, 'data', 'processed')
interimPath = os.path.join(PROJ_ROOT, 'data', 'interim')
df = pd.read_csv(os.path.join(procPath, "player_bios_processed.csv"))

In [25]:
df.columns

Index([u'Student_ID', u'Name', u'City', u'High School', u'Hometown', u'Ht.',
       u'No.', u'Position', u'Region', u'Wt.', u'Active Seasons', u'Misc',
       u'Bio', u'College'],
      dtype='object')

In [31]:
df['Hometown']

0                          Bonne Terre, MO / North County
1                      Seattle, Wash. / South Kent School
2                     Buena Park, Calif. / Vista Murrieta
3                   Altamonte Springs, FL / Lake Brantley
4       Port Washington, N.Y. / Paul D. Schreiber High...
5              Somerset West, South Africa / Panel Vallei
6          Eden Prairie, Minn. / Eden Prairie High School
7          St. Catharines, Ont., Canada / Governor Simcoe
8                                   Cypress, TX / Cy-Fair
9                                                   - / -
10                       Miramar, Fla. / Somerset Academy
11               Barnstable, MA / Phillips Exeter Academy
12                 Eagle River, Alaska / Brewster Academy
13                            Dover, NH / Phillips Exeter
14                          Olney, Md. / Montgomery Blair
15                               Richmond, Va. / Deep Run
16               Newton, Mass. / Newton North High School
17            

In [42]:
places = df['Hometown'].apply(lambda x: x.split("/")[0].strip() if type(x )== str else "")

In [43]:
places.shape

(7274L,)

In [48]:
uplaces.shape

(3274L,)

In [45]:
uplaces = places.unique()

In [46]:
uOut = os.path.join(interimPath, 'hometowns.csv')

In [47]:
pd.Series(uplaces).to_csv(uOut)

In [ ]:
# use chmod +x first to make script executable, then runsrc/data/geocode
# if gnu parallel not available, 
# currently uses cameron's mapquest API key, get 15k calls per month
# csvcut hometowns.csv -c 2  | parallel --progress ./geocode {} > geocodes.csv

In [54]:
!ls ../data/interim

harvard_bio_parsed.csv.gz
harvard_bios_processed.csv
harvard_players.json
harvard_rosters.p
hometowns.csv
players.json
rosters.p
yale_bio_parsed.csv.gz
yale_bios_processed.csv


In [56]:
biosPath = os.path.join(PROJ_ROOT, 'data', 'interim', "yale_bio_parsed.csv.gz")

In [57]:
d2 = pd.read_csv(biosPath, compression='gzip')

In [66]:
d2['synopsis'].iloc[0][:8]

'Row(_1=u'

In [60]:
d2['synopsis']

0       Row(_1=u'\n2009: Made 22 appearances, includin...
1       Row(_1=u'\n2010: Team captain.\n2009: Made 12 ...
2       Row(_1=u'\n2015-16 (Junior): Three\xc2\xa0seat...
3       Row(_1=u'\n2012-13:\xa0Rowed in stroke-seat of...
4       Row(_1=u"\n2006 Backup DB who played in all bu...
5       Row(_1=u'\n2008: First-team All-Ivy... Started...
6       Row(_1=u'\n2008: Honorable-mention All-Ivy... ...
7       Row(_1=u'\n2008: Student assistant coach with ...
8       Row(_1=u'\n2008: Earned varsity letter after a...
9       Row(_1=u'\n2007: Two touchdowns at rival Princ...
10      Row(_1=u'\n2008: Started seven of ten game at ...
11      Row(_1=u'\n2006: Recovered from injury and did...
12      Row(_1=u'\n2013: Made 16 appearances with four...
13      Row(_1=u'\n2006: Varsity backup TE who played ...
14      Row(_1=u"\n2010-11:  Rowed in the bow seat of ...
15      Row(_1=u'\n2008: Played in all 10 games on spe...
16      Row(_1=u'\n2008: Missed season due to injury.\...
17      Row(_1